# LLM Zoomcamp HW3

## Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("multi-qa-distilbert-cos-v1")

/home/codespace/.python/current/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [119]:
user_question = "I just discovered the course. Can I still join it?"

result = embedding_model.encode(user_question)

In [120]:
result[0]

0.078222655

## Prepare the documents

In [5]:
import requests

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/documents-with-ids.json"
docs_url = f"{base_url}/{relative_url}?raw=1"
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [16]:
docs_ml = []
for doc in documents:
    if doc["course"] == "machine-learning-zoomcamp":
        docs_ml.append(doc)

In [17]:
len(docs_ml)

375

In [18]:
docs_ml[1]

{'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
 'section': 'General course-related questions',
 'question': 'Is it going to be live? When?',
 'course': 'machine-learning-zoomcamp',
 'id': '39fda9f0'}

## Q2. Creating the embeddings

In [19]:
import numpy as np
from tqdm.auto import tqdm

embeddings = []

for doc in tqdm(docs_ml):
    question = doc["question"]
    text = doc["text"]
    qa_text = f"{question} {text}"
    embedding_qa_text = embedding_model.encode(qa_text)
    embeddings.append(embedding_qa_text)

X = np.array(embeddings)
X.shape

  0%|          | 0/375 [00:00<?, ?it/s]

(375, 768)

In [150]:
X[1]

array([ 7.06691891e-02, -1.93056818e-02,  7.73837268e-02,  2.37205159e-02,
        8.63914639e-02, -2.04659123e-02,  9.15529486e-03,  4.65581082e-02,
       -4.78188833e-03, -1.43747935e-02, -1.97507888e-02, -4.51360121e-02,
        3.47972587e-02, -4.37994339e-02,  3.87682468e-02, -2.01997068e-02,
       -1.13333657e-03, -4.24227193e-02,  3.04053314e-02, -3.29325274e-02,
       -8.21831822e-03,  2.65472522e-03,  3.13991532e-02,  4.64935750e-02,
       -4.02437896e-02,  5.76140992e-02, -1.99844502e-02,  2.95837503e-02,
       -8.68471432e-03, -9.97345243e-03,  7.24730035e-03, -5.11227474e-02,
        7.30461255e-02,  2.48600785e-02,  4.75398563e-02, -1.94741841e-02,
       -6.19658045e-02,  6.10636733e-02, -7.48167513e-03,  4.05436046e-02,
       -3.07619609e-02,  1.59537047e-02,  2.08376385e-02,  1.18389828e-02,
       -3.59451994e-02, -1.50297992e-02,  4.09365706e-02, -4.85859904e-03,
       -3.44599858e-02,  3.27185616e-02, -1.14695383e-02,  4.67685349e-02,
        8.60479251e-02,  

## Q3. Search

In [20]:
v = result

In [24]:
scores = X.dot(v)
np.max(scores)

0.6506573

## Vector search

In [86]:
class VectorSearchEngine:
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'You can find the latest and up-to-date deadlines here: https://docs.google.com/spreadsheets/d/e/2PACX-1vQACMLuutV5rvXg5qICuJGL-yZqIV0FBD84CxPdC5eZHf8TfzB-CJT_3Mo7U7oGVTXmSihPgQxuuoku/pubhtml\nAlso, take note of Announcements from @Au-Tomator for any extensions or other news. Or, the form may also show the updated deadline, if Instructor(s) has updated it.',
  'section': 'General course-related questions',
  'question': 'Homework - What are homework and project deadlines?',
  'course': 'data-engineering-zoomcamp',
  'id': 'a1daf537'},
 {'text': 'After you submit your homework it will be graded based on the amount of questions in a particular homework. You can see how many points you have right on the page of the homework up top. Additionally in the leaderboard you will find the sum of all points you’ve earned - points for Homeworks, FAQs and Learning in Public. If homework is clear, others work as follows: if you submit something to FAQ, you get one point, for each learning i

## Q4. Hit-rate for our search engine

In [98]:
import pandas as pd

base_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main"
relative_url = "03-vector-search/eval/ground-truth-data.csv"
ground_truth_url = f"{base_url}/{relative_url}?raw=1"

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == "machine-learning-zoomcamp"]
ground_truth = df_ground_truth.to_dict(orient="records")

In [99]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [100]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [108]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [109]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q["document"]
        results = search_function(q)
        relevance = [d["id"] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        "hit_rate": hit_rate(relevance_total),
        "mrr": mrr(relevance_total),
    }

In [106]:
def question_text_vector_custom_search(q):
    question = q["question"]
    v_q = embedding_model.encode(question)

    return VectorSearchEngine(documents=docs_ml, embeddings=X).search(
        v_q, num_results=5
    )

In [107]:
evaluate(ground_truth, question_text_vector_custom_search)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

## Q5. Indexing with Elasticsearch

In [115]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine",
            },
        }
    },
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [140]:
# Insert vector to each document entry
for vector in range(len(X)):
    docs_ml[vector]["question_text_vector"] = X[vector]

In [151]:
docs_ml[1]

{'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
 'section': 'General course-related questions',
 'question': 'Is it going to be live? When?',
 'course': 'machine-learning-zoomcamp',
 'id': '39fda9f0',
 'question_text_vector': array([ 7.06691891e-02, -1.93056818e-02,  7.73837268e-02,  2.37205159e-02,
         8.63914639e-02, -2.04659123e-02,  9.15529486e-03,  4.65581082e-02,
        -4.78188833e-03, -1.43747935e-02, -1.97507888e-02, -4.51360121e-02,
         3.47972587e-02, -4.37994339e-02,  3.87682468e-02, -2.01997068e-02,
        -1.13333657e-03, -4.24227193e-02,  3.04053314e-02, -3.29325274e-02,
        -8.21831822e-03,  2.65472522e-03,  3.13991532e-02,  4.64935750e-02,
        -4.02437896e-02, 

In [144]:
for doc in tqdm(docs_ml):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/375 [00:00<?, ?it/s]

In [145]:
query = {
    "field": "question_text_vector",
    "query_vector": result,
    "k": 5,
    "num_candidates": 10000,
}

In [148]:
res = es_client.search(
    index=index_name, knn=query, source=["text", "section", "question", "course", "id"]
)

res["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': '6NP1m5ABusD-agPuioUb',
 '_score': 0.82532895,
 '_source': {'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'}}

## Q6. Hit-rate for Elasticsearch

In [152]:
def elastic_search_knn(vector):
    knn = {
        "field": "question_text_vector",
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {"term": {"course": "machine-learning-zoomcamp"}},
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"],
    }

    es_results = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in es_results["hits"]["hits"]:
        result_docs.append(hit["_source"])

    return result_docs

In [155]:
def question_text_vector_knn(q):
    question = q["question"]

    v_q = embedding_model.encode(question)

    return elastic_search_knn(v_q)


evaluate(ground_truth, question_text_vector_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}